# Wrappers: Bridge to External Libraries

This tutorial covers the `Wrapper` composition pattern in MolPy and demonstrates the `RDKitWrapper` adapter with runnable examples.

**Learning objectives**
- Understand the purpose and design of `Wrapper` (composition-based integration).
- Use `RDKitWrapper` to bridge MolPy `Atomistic` objects and RDKit `Mol` objects.
- Perform bidirectional coordinate synchronization and use RDKit cheminformatics tools alongside MolPy operations.

## 1. Wrapper pattern (concept)

A `Wrapper` in MolPy is a composition-based adapter: it *holds* a MolPy `Atomistic` (or similar) and an external library object (e.g., an RDKit `Mol`), providing bidirectional synchronization and unified access.

Key expectations:
- `Wrapper` stores an internal MolPy object (accessible as `wrapper.inner`).
- It also stores an external object (e.g., `wrapper.mol` for RDKit).
- Provide factory methods (e.g., `from_smiles()`, `from_atomistic()`) and `sync` methods for coordinates/data.

## 3. RDKitWrapper: basic usage examples

The examples below use `RDKitWrapper` to create wrappers from SMILES and from an existing MolPy `Atomistic`.

In [ ]:
import numpy as np
import molpy as mp
mol = Chem.MolFromSmiles('CCO')  # ethanol
wrapper = mp.adapter.RDKitWrapper.from_mol(mol)
print('MolPy Atomistic access: inner or core ->', type(wrapper.inner))
print('Number of atoms (MolPy):', getattr(wrapper, 'n_atoms', getattr(wrapper.inner, 'n_atoms', 'unknown')))
rdkit_mol = wrapper.mol
print('RDKit Mol atoms:', rdkit_mol.GetNumAtoms())
print('Can unwrap to get native Atomistic:', wrapper.unwrap() is wrapper.inner)


MolPy Atomistic access: inner or core -> <class 'molpy.core.atomistic.Atomistic'>
Number of atoms (MolPy): unknown
RDKit Mol atoms: 3
Can unwrap to get native Atomistic: True


## 4. Coordinate synchronization (bidirectional)

Use `sync_coords_from_mol()` to copy coordinates *from* RDKit into MolPy, and `sync_coords_to_mol()` to copy from MolPy into RDKit. Below: generate a 3D conformation in RDKit, then sync it into MolPy and back.

In [ ]:
# Generate 3D in RDKit, sync to MolPy, modify, and sync back
mol = Chem.MolFromSmiles('c1ccccc1O')  # phenol
w = RDKitWrapper.from_mol(mol)
print('Before 3D: RDKit has', w.mol.GetNumConformers(), 'conformers')
# Use wrapper.generate_3d which handles AddHs, embedding, optimization and mapping
w.generate_3d(optimize=True, random_seed=1)
print('After generate_3d: RDKit has', w.mol.GetNumConformers(), 'conformers')
# Sync RDKit coords -> MolPy (generate_3d already transferred coordinates/hydrogens)
print('Synced RDKit -> MolPy; MolPy coords shape:', getattr(w.inner, 'coords', 'no-coords'))
# Simple MolPy-side op: translate center to origin.
# RDKitWrapper transfers coordinates into each atom as the 'xyz' key,
# so read coordinates directly from per-atom data and normalize to ndarray.

coords = np.array([a.get('xyz', [0.0, 0.0, 0.0]) for a in w.inner.atoms])
c = coords.mean(axis=0)
# Write back: prefer bulk assignment if supported, otherwise per-atom 'xyz'
new_coords = coords - c
try:
    w.inner.coords = new_coords
    print('Centered MolPy coordinates')
except Exception:
    for i, a in enumerate(w.inner.atoms):
        a['xyz'] = new_coords[i].tolist()
    print('Centered MolPy coordinates (per-atom)')
# Sync back to RDKit
w.sync('to_mol')
print('Synced MolPy -> RDKit; RDKit conformer updated')


Before 3D: RDKit has 0 conformers
After generate_3d: RDKit has 1 conformers
Synced RDKit -> MolPy; MolPy coords shape: no-coords
Centered MolPy coordinates
Synced MolPy -> RDKit; RDKit conformer updated


## 5. RDKit cheminformatics examples (SMILES/SMARTS)

Use RDKit through the wrapper to compute properties and perform substructure searches.

In [26]:
# SMILES canonicalization and SMARTS matching via RDKit
mol = Chem.MolFromSmiles('CC(=O)Oc1ccccc1C(=O)O')  # aspirin-like
w = RDKitWrapper.from_mol(mol)
smi = Chem.MolToSmiles(w.mol)
print('Canonical SMILES:', smi)
# Substructure: benzene ring
benz = Chem.MolFromSmarts('c1ccccc1')
if w.mol.HasSubstructMatch(benz):
    print('Contains benzene ring - matches:', len(w.mol.GetSubstructMatches(benz)))
print('Mol weight (RDKit):', Descriptors.MolWt(w.mol))


Canonical SMILES: CC(=O)Oc1ccccc1C(=O)O
Contains benzene ring - matches: 1
Mol weight (RDKit): 180.15899999999996


## 6. Practical workflow example

A short end-to-end workflow combining MolPy and RDKit: create from SMILES, generate 3D, sync, do a MolPy op, sync back, compute properties.

In [27]:
from rdkit.Chem import AllChem, Descriptors
import numpy as np

# 1. Start from SMILES
mol = Chem.MolFromSmiles('CCCCCC')  # Hexane
w = RDKitWrapper.from_mol(mol)
print('Step 1: Created from SMILES')

# 2. Generate 3D via wrapper helper
w.generate_3d(optimize=True, random_seed=42)
print(f'Step 2: Generated 3D structure with {w.mol.GetNumAtoms()} atoms')

# 3. Sync coordinates to MolPy
w.sync('from_mol')
atomistic = w.inner
print('Step 3: Synced to MolPy Atomistic')

# 4. Use MolPy operations (e.g., center molecule)
# RDKitWrapper stores coordinates on each atom as the 'xyz' key,
# so we read per-atom coords directly and normalize to a numpy array.
coords = np.array([a.get('xyz', [0.0, 0.0, 0.0]) for a in atomistic.atoms])
center = coords.mean(axis=0)
new_coords = coords - center
try:
    atomistic.coords = new_coords
    print('Step 4: Centered molecule at origin')
except Exception:
    for i, a in enumerate(atomistic.atoms):
        a['xyz'] = new_coords[i].tolist()
    print('Step 4: Centered molecule by per-atom xyz')

# 5. Sync back to RDKit for further analysis
w.sync('to_mol')
print('Step 5: Synced back to RDKit Mol')

# 6. Calculate properties
mw = Descriptors.MolWt(w.mol)
logp = Descriptors.MolLogP(w.mol)
print('\nFinal properties:')
print(f'  Molecular weight: {mw:.2f} g/mol')
print(f'  LogP: {logp:.2f}')


Step 1: Created from SMILES
Step 2: Generated 3D structure with 20 atoms
Step 3: Synced to MolPy Atomistic
Step 4: Centered molecule at origin
Step 5: Synced back to RDKit Mol

Final properties:
  Molecular weight: 86.18 g/mol
  LogP: 2.59


## 7. Creating custom Wrappers

If you need to support another external library, inherit from `Wrapper` and implement factory and sync methods. Below is a minimal skeleton.

In [28]:
# Minimal custom wrapper skeleton (non-runnable template)
from molpy.core.wrappers.base import Wrapper

class MyExternalWrapper(Wrapper):
    """Example skeleton for integrating another library.
    Implement `from_external`, `sync_to_external`, and `sync_from_external`.
    """
    def __init__(self, atomistic, external_obj):
        super().__init__(atomistic)
        self.external_obj = external_obj

    @classmethod
    def from_external(cls, external_obj):
        # convert external_obj -> Atomistic (user implements)
        atomistic = ...
        return cls(atomistic, external_obj)

    def sync_to_external(self):
        # copy MolPy -> external_obj
        raise NotImplementedError()

    def sync_from_external(self):
        # copy external_obj -> MolPy
        raise NotImplementedError()


## Summary & Next steps

- `Wrapper` is a composition-based adapter for external libraries.
- `RDKitWrapper` demonstrates common tasks: creation from SMILES/Atomistic, coordinate sync, and using RDKit tools alongside MolPy.

Next: run the cells in order. If you want, I can run the notebook cells here to validate execution and fix any runtime errors (e.g., missing RDKit). Would you like me to run them now?